## Nota: Descomprimir el archivo PROFECO_2015_vEspecial.csv en el folder import de Node4j y poner este notebook ahí mismo.

In [1]:
import pandas as pd

El CSV viene con un encoding ISO-8859-1, por lo que primero lo pasamos a UTF-8

In [2]:
df = pd.read_csv("Datos/PROFECO_2015_vEspecial.csv",encoding='ISO-8859-1')
df.head(2)

,AÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂO,UNIDAD ADMINISTRATIVA,NUMERO DE EXPEDIENTE DE VERIFICACION,FECHA DE VISITA,NORMA OFICIAL MEXICANA VERIFICADA,TIPO DE PRODUCTO,DESCRIPCION DEL PRODUCTO,RAZON SOCIAL VISITADA,GIRO COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,CLASIFICACION RESULTADO PRODUCTO
0,2015,AGUASCALIENTES,702/000009/2015,2015-01-05,NOM-015-SCFI-2007,MUÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂECOS,EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGI...,JULIO CEPEDA JUGUETERIAS S.A. DE C.V.,JUGUETERIA,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
1,2015,AGUASCALIENTES,702/000012/2015,2015-01-06,NOM-004-SCFI-2006,ROPA CABALLERO,CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE ...,C&A DE MEXICO S. DE R.L.,TIENDA DEPARTAMENTAL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA


In [3]:
df.to_csv('PROFECO_2015_vEspecial.csv', index=False, encoding='utf-8')

In [4]:
from neo4j import GraphDatabase

Ponemos nuestro usuario y contraseña:

In [5]:
usuario = 'neo4j'
contra = 'neo4j' 

In [6]:
#Genero Driver. NOTA: Dejar el puerto como esta, es otra conexión que el web portal
driver = GraphDatabase.driver("bolt://localhost:7687", auth=(usuario, contra), encrypted = False)
#Eliminamos constraints existentes
with driver.session() as s:
    # Drop constraints / indices
    for c in s.run("CALL db.constraints"):
        s.run("DROP CONSTRAINT " + c[0])
    for c in s.run("CALL db.indexes"):
        s.run("DROP INDEX " + c[1])
    s.run("MATCH (n) DETACH DELETE n")

Cargamos CSV a la base

In [7]:
with driver.session() as s:
    s.run("CREATE CONSTRAINT ON (e:Entidad) ASSERT e.nombre IS UNIQUE;")
    s.run("CREATE INDEX ON :Municipio (nombre);")
    s.run("CREATE INDEX ON :Municipio (nombre, entidad);")
    s.run("CREATE INDEX ON :Tienda (nombre);")
    s.run("CREATE INDEX ON :Tienda (nombre, giro);")
    s.run("CREATE INDEX ON :Producto (nombre);")
    s.run("CREATE INDEX ON :Producto (nombre, categoria);")
    s.run("CREATE CONSTRAINT ON (c:Categoria) ASSERT c.nombre IS UNIQUE;")
    s.run("CREATE INDEX ON :Usuario (id);")
    s.run("CREATE INDEX ON :Compra (id);")
    print("Indices creados.")
    s.run("""LOAD CSV WITH HEADERS FROM "file:///PROFECO_2015_vEspecial.csv" AS row
             MERGE (e:Entidad {nombre: row.ENTIDAD})
             MERGE (m:Municipio {nombre: row.MUNICIPIO, entidad: row.ENTIDAD})
             MERGE (e) <-[r1:Es_Municipio]- (m);
             """)
    print("Entidades/Municipios y sus relaciones creadas.")
    s.run("""LOAD CSV WITH HEADERS FROM "file:///PROFECO_2015_vEspecial.csv" AS row
             MERGE (t:Tienda {nombre: row.`RAZON SOCIAL VISITADA`, giro: row.`GIRO COMERCIAL`, municipio:row.MUNICIPIO , entidad: row.ENTIDAD});
             """)
    print("Tiendas creadas.")
    s.run("""MATCH (t:Tienda), (m: Municipio)
             WHERE t.municipio = m.nombre and t.entidad = m.entidad
             CREATE (t) -[r:Se_Encuentra_En]-> (m);
             """)
    print("Relaciones entre Tiendas y Municipios creadas.")
    s.run("""LOAD CSV WITH HEADERS FROM "file:///PROFECO_2015_vEspecial.csv" AS row
             MERGE (c:Categoria {nombre: row.`TIPO DE PRODUCTO`});
             """)
    print("Categorías creadas.")
    s.run("""LOAD CSV WITH HEADERS FROM "file:///PROFECO_2015_vEspecial.csv" AS row
             MERGE (p:Producto {nombre: row.`DESCRIPCION DEL PRODUCTO` ,categoria: row.`TIPO DE PRODUCTO`});
             """)
    print("Productos creados.")
    s.run("""MATCH (p:Producto), (c:Categoria)
             WHERE p.categoria = c.nombre
             CREATE (p) -[r:Categorizado_Como]-> (c);
             """)
    print("Relaciones entre Categorías y Productos creadas.")
    s.run("""LOAD CSV WITH HEADERS FROM "file:///PROFECO_2015_vEspecial.csv" AS row
             MATCH (e:Entidad {nombre: row.ENTIDAD}) <-[r1:Es_Municipio]- (m:Municipio {nombre: row.MUNICIPIO}) <-[r2:Se_Encuentra_En]- (t:Tienda {nombre: row.`RAZON SOCIAL VISITADA`, giro: row.`GIRO COMERCIAL`})
             MATCH (p:Producto {nombre: row.`DESCRIPCION DEL PRODUCTO`}) -[r:Categorizado_Como]-> (c:Categoria {nombre: row.`TIPO DE PRODUCTO`})
             CREATE (t) -[r3:Vende {incumplimiento: row.INCUMPLIMIENTO}]-> (p);
             """)
    print("Relaciones entre Tiendas y Productos creadas.")

Indices creados.
Entidades/Municipios y sus relaciones creadas.
Tiendas creadas.
Relaciones entre Tiendas y Municipios creadas.
Categorías creadas.
Productos creados.
Relaciones entre Categorías y Productos creadas.
Relaciones entre Tiendas y Productos creadas.


In [8]:
flatten = lambda l: [item for sublist in l for item in sublist]

lista_productos = []
with driver.session() as s:
    lista_productos = flatten(s.run("MATCH (n:Producto) return n.nombre;").values())

In [9]:
lista_productos

['EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGIDO  MUÃ\x91ECA PARA NIÃ\x91A DE LA MARCA BARBIE DE ORIGEN CHINA. CONTENIDO UNA PIEZA MODELO BARBIE BASICA LIFE FASHION CON NUMERO DE LOTE T7439 INDICA INFORMACION IMPORTANTE PARA CONSULTA  IMPO',
 'CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE LA MARCA C & A DE ORIGEN BANGLADESH  IMPORTADO POR C & A MEXICO  DE COMPOSICION EN 64% ALGODON  32% POLIAMIDA Y 4% ELASTANO CON NUMERO DE LOTE 252959. CONTIENE INSTRUCCIONES DE CUIDADO',
 'UNA BOTELLA BRANDY DE LA MARCA TORRES 10 CON LA LEYENDA 100% DE UVA  GRAN RESERVA CON 38% ALC. VOL. CON UN CONTENIDO NETO DE 700ML CON SELLO DE SHCP CON LA LEYENDA "EL ABUSE EN EL CONSUMO DE ESTE PRODUCTO ES NOCIVO PARA LA SALUD"',
 'ENVASE DE VIDRIO CONTENIDO BRANDY DE LA MARCA AZTECA DE ORO  SOLERA RESERVADA CON 38% ALC. VOL. CONTENIDO NETO 700ML HECHO ENMEXICO PRODUCTO 100% DE UVA FABRICADO POR INDUSTRIAS VINICOLAS PEDRO DOMER  S.A. DE C.V.',
 'BOTELLA DE VIDRIO CONTENIDO BRANDY 100% UVA GRAN RESERVA MARCA TORR

# Consultas

## Dado un estado y un producto, buscar lugares donde pueda encontrarlo.

In [10]:
def buscaLugares(entidad, producto, numMax = 10):
    with driver.session() as s:
        consulta = s.run("""MATCH (n:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                 WHERE n.nombre = \"{0}\" AND e.nombre = \"{1}\"
                 RETURN t
                 LIMIT {2};
                 """.format(producto, entidad, numMax)).values()
                
    # Regresemos un dataframe para ver el resultado de la consulta claramente        
    
    data_consulta = pd.DataFrame(columns = ['Estado', 'Producto', 'Tienda'])
    
    
    for i,x in enumerate(consulta):
        properties = x[0]._properties
        names = [properties['entidad'], producto, properties['nombre']]
        data_consulta.loc[i] = names
        
    return data_consulta[0:10]

In [11]:
buscaLugares("CHIHUAHUA","huevo")

,Estado,Producto,Tienda
0,CHIHUAHUA,huevo,OPERADORA DE CD JUAREZ SA DE CV
1,CHIHUAHUA,huevo,TIENDAS SORIANA S.A. DE C.V.
2,CHIHUAHUA,huevo,SUPER MERCADO GONZALEZ EXPRESS


## Dado un estado y una tienda, verificar si tiene algun incumplimiento con un producto.

In [12]:
def revisaIncumplimiento(entidad, tienda):
    with driver.session() as s:
        consulta = bool(s.run("""MATCH (n:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                     WHERE t.nombre = \"{1}\" AND e.nombre = \"{0}\" AND r1.incumplimiento <> \"NO SE DETECTO INCUMPLIMIENTO\"
                     RETURN count(*);
                     """.format(entidad,tienda)).value()[0])
        
    if not consulta:
        return'No se detecto incumplimiento'
    else:
        return 'Se detectó incumplimiento'

In [13]:
revisaIncumplimiento("CAMPECHE","CARLA VALLADARES CARAVEO Y/O ULTRAMART HOSPITALES")

'No se detecto incumplimiento'

## Dado un estado y un producto, buscar alternativas sin incumplimiento de ese producto o categoría.

In [14]:
def buscaLugaresLimpios(entidad, producto, buscarCategoria = False, numMax = 10):
    with driver.session() as s:
        if buscarCategoria == True:
            consulta = s.run("""MATCH (n:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                            MATCH (n) -[r4:Categorizado_Como]-> (c:Categoria)
                            MATCH (c) <-[r5:Categorizado_Como]- (p_alt:Producto) <-[r6:Vende]- (t2:Tienda) -[r7:Se_Encuentra_En]-> (m2:Municipio) -[r8:Es_Municipio]-> (e)
                            WHERE n.nombre = \"{0}\" AND r6.incumplimiento = \"NO SE DETECTO INCUMPLIMIENTO\" AND e.nombre = \"{1}\"
                            RETURN p_alt, t2
                            LIMIT 10
                            """.format(producto, entidad, numMax)).values()
        else:
            #Quiero buscar productos alternativos
            consulta =  s.run("""MATCH (n:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                                WHERE n.nombre = \"{0}\" AND r1.incumplimiento = \"NO SE DETECTO INCUMPLIMIENTO\" AND e.nombre = \"{1}\"
                                RETURN n.nombre, t
                                LIMIT 10
                                """.format(producto, entidad, numMax)).values()
            
        if buscarCategoria:
            data_consulta = pd.DataFrame(columns = ['Producto', 'Categoría', 'Tienda'])
    
    
            for i,x in enumerate(consulta):
                prod = x[0]._properties
                tienda = x[1]._properties
                names = [prod['nombre'], prod['categoria'], tienda['nombre']]
                data_consulta.loc[i] = names
                
            return data_consulta.loc[0:10]
        
        else:
            data_consulta = pd.DataFrame(columns = ['Producto', 'Tienda'])
    
    
            for i,x in enumerate(consulta):
                tienda = x[1]._properties
                names = [producto, tienda['nombre']]
                data_consulta.loc[i] = names
                
            return data_consulta.loc[0:10]
                      

In [15]:
(buscaLugaresLimpios("CAMPECHE","lomo de atun aleta amarilla", buscarCategoria = False))

,Producto,Tienda
0,lomo de atun aleta amarilla,CARLA VALLADARES CARAVEO Y/O ULTRAMART HOSPITALES


In [16]:
buscaLugaresLimpios("CAMPECHE", "lomo de atun aleta amarilla", buscarCategoria = True)[0:3]

,Producto,Categoría,Tienda
0,si cumple norma,ATUN,CADENA COMERCIAL OXXO SA DE CV YO TIENDA DE CO...
1,si cumple con la norma lata de lomo de atun d...,ATUN,ULTRAMARINOS EL SOL Y/O KRISHNA HUMBERTO SANDO...
2,de 140 grs aleta amarilla en aceite,ATUN,SUPER WILLY'S SA DE CV


## Llevar un registro de las compras y lugares que ha hecho cada usuario.

In [17]:
import random

In [18]:
def get_random_alphaNumeric_string(stringLength=8):
    lettersAndDigits = string.ascii_letters + string.digits
    return ''.join((random.choice(lettersAndDigits) for i in range(stringLength)))


#Sistema de usuarios
def agregaCompraUsuario(usuario_id, nombre_usuario, compra_id, producto, cantidad, lugar, municipio, entidad):
    #with driver.session() as s:
    with driver.session() as s:
        return s.run("""MATCH (p:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                 WHERE p.nombre = \"{0}\" AND t.nombre = \"{1}\" AND m.nombre = \"{2}\" AND e.nombre = \"{3}\"
                 MERGE (u:Usuario {{id: \"{4}\"}}) -[r4:Realiza]-> (c:Compra {{id: \"{5}\"}})
                 ON CREATE SET u.nombre =  \"{6}\", c.hora = timestamp()
                 MERGE (c) -[r5:Contiene {{cantidad: \"{7}\"}}]-> (p)
                 MERGE (c) -[r6:En]-> (t)
                """.format(producto, lugar, municipio, entidad, usuario_id, compra_id, nombre_usuario ,cantidad))
def consultarProductosUsuario(usuario_id):
    with driver.session() as s:
        return s.run("""MATCH (u:Usuario) -[r1:Realiza]-> (c:Compra) -[r2:Contiene]-> (p:Producto)
                 WHERE u.id = \"{0}\"
                 RETURN DISTINCT p.nombre
                 """.format(usuario_id)).values()

def consultarLugaresUsuario(usuario_id):
    with driver.session() as s:
        return s.run("""MATCH (u:Usuario) -[r1:Realiza]-> (c:Compra) -[r2:En]-> (t:Tienda) -[r3:Se_Encuentra_En]-> (m:Municipio) -[r4:Es_Municipio]-> (e:Entidad)
                 WHERE u.id = \"{0}\"
                 RETURN DISTINCT t.nombre, m.nombre, e.nombre
                 """.format(usuario_id)).values()

Ahora agreguemos algunos usuarios:

In [37]:
agregaCompraUsuario("usuario1","Alejandro","compra1","",
                    3,"CARLA VALLADARES CARAVEO Y/O ULTRAMART HOSPITALES","CAMPECHE","CAMPECHE")
agregaCompraUsuario("usuario1","Alejandro","compra2","cono de 12 piezas de la marca crio.",
                    1,"CADENA COMERCIAL OXXO SA DE CV Y/O SUC BOSQUES DE CAMPECHE","CAMPECHE","CAMPECHE")
agregaCompraUsuario("usuario1","Alejandro","compra3","bolsa de arroz lote 1014",
                    5,"CADENA COMERCIAL OXXO SA DE CV","CAMPECHE","CAMPECHE")
agregaCompraUsuario("usuario1","Alejandro","compra12","BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.",
                    1,"CADENA COMERCIAL OXXO SA DE CV","AGUASCALIENTES","AGUASCALIENTES")

agregaCompraUsuario("usuario2","Mario","compra4","BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.",
                    1,"CADENA COMERCIAL OXXO SA DE CV","AGUASCALIENTES","AGUASCALIENTES")
agregaCompraUsuario("usuario2","Mario","compra5","BOTELLA DE BRANDY CONTENIDO NETO 700 ML 38% ALC.",
                    1,"CADENA COMERCIAL OXXO SA DE CV","AGUASCALIENTES","AGUASCALIENTES")
agregaCompraUsuario("usuario2","Mario","compra6","BOTELLA DE BRANDY CONTENIDO NETO 700 ML 38% ALC.",
                    1,"CADENA COMERCIAL OXXO SA DE CV","AGUASCALIENTES","AGUASCALIENTES")
agregaCompraUsuario("usuario2","Mario","compra7","BOTELLA DE VIDRIO CON TEQUILA REPOSADO 100% DE AGAVE AZUL  CONTENIDO NETO DE 695 ML CON 38% ALC.",
                    1,"TIENDAS EXTRA S.A DE C.V","AGUASCALIENTES","AGUASCALIENTES")

agregaCompraUsuario("usuario3","David","compra8","Pinza de presion marca MAX TOOL modelo 306701 en blister conteniendo 1 pza",
                    2,"TLAPALERIA  HIDROMECANICA","COYOACAN","DISTRITO FEDERAL")
agregaCompraUsuario("usuario3","David","compra9","Desarmador de cruz marca MAX TOOL modelo 300201 en blister conteniendo 1 pza",
                    1,"TLAPALERIA  HIDROMECANICA","COYOACAN","DISTRITO FEDERAL")
agregaCompraUsuario("usuario3","David","compra10","Brocha marca MAX TOOL modelo 301507 en blister",
                    1,"TLAPALERIA  HIDROMECANICA","COYOACAN","DISTRITO FEDERAL")
agregaCompraUsuario("usuario3","David","compra11","BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.",
                    1,"CADENA COMERCIAL OXXO SA DE CV","AGUASCALIENTES","AGUASCALIENTES")

In [20]:
consultarProductosUsuario("usuario2")

[['BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.'],
 ['BOTELLA DE BRANDY CONTENIDO NETO 700 ML 38% ALC.'],
 ['BOTELLA DE VIDRIO CON TEQUILA REPOSADO 100% DE AGAVE AZUL  CONTENIDO NETO DE 695 ML CON 38% ALC.']]

In [21]:
consultarLugaresUsuario("usuario1")

[['CADENA COMERCIAL OXXO SA DE CV', 'CAMPECHE', 'CAMPECHE'],
 ['CADENA COMERCIAL OXXO SA DE CV', 'AGUASCALIENTES', 'AGUASCALIENTES']]

## Recomendar a un usuario una tienda en donde pueda encontrar en un solo lugar, lo que compra en diferentes tiendas.

In [22]:
def recomendar_lugar(productos, entidad):
    
    condicion = ''

    for x in productos:
        condicion = condicion + 'n.nombre = \"{}\"'.format(x) + ' OR '
        
    condicion = condicion[0:-4]
    
    with driver.session() as s:
        consulta =  s.run("""MATCH (n:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                     WHERE e.nombre = \"{0}\" AND ({1})
                     RETURN t.nombre, n.nombre
                     ORDER BY t.nombre
                     """.format(entidad,condicion)).values()
        
    data_consulta = pd.DataFrame(consulta, columns = ['Tienda', 'Producto'])
    data_consulta = data_consulta.groupby('Tienda')['Producto'].apply(lambda x: ';'.join(x)).reset_index()
    data_consulta['Producto'] = data_consulta['Producto'].apply(lambda x: x.split(';'))

    condicion = [True if set(productos) == set(x) else False for x in data_consulta['Producto']]

    if data_consulta[condicion].shape[0] > 0:
        return data_consulta[condicion][['Tienda']].reset_index(drop=True).loc[0:5]
    else:
        print('No hay tiendas')
        return []


In [23]:
productos = ['BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.', 'BOTELLA DE BRANDY CONTENIDO NETO 700 ML 38% ALC.',
            'BOTELLA DE WHISKY CONENIDO 750 ML CON 40% ALC.']
entidad = 'AGUASCALIENTES'

print(recomendar_lugar(productos, entidad))

                           Tienda
0  CADENA COMERCIAL OXXO SA DE CV


## Encontrar los estados con mayor y menor incumplimiento relativo al número de tiendas que tiene.

In [24]:
def encuentraIncumplimientoEstados():
    with driver.session() as s:
        incump = s.run("""MATCH (n:Producto) <-[r1:Vende]- (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                        WHERE r1.incumplimiento <> \"NO SE DETECTO INCUMPLIMIENTO\"
                        RETURN e.nombre, count(r1)
                        """).values()
        totals = s.run("""MATCH (t:Tienda) -[r2:Se_Encuentra_En]-> (m:Municipio) -[r3:Es_Municipio]-> (e:Entidad)
                        RETURN e.nombre, count(t)
                        """).values()
        lista = []
        for k,v in totals:
            num_inc = 0
            for k2,v2 in incump:
                if k == k2:
                    num_inc=v2
                    break
            lista.append([k,num_inc/v])
        df_ret = pd.DataFrame(lista, columns = ['Entidad','Incump_Por_Tiendas'])
    return df_ret

In [25]:
print(encuentraIncumplimientoEstados().head())

               Entidad  Incump_Por_Tiendas
0       AGUASCALIENTES            0.194175
1      BAJA CALIFORNIA            0.174051
2  BAJA CALIFORNIA SUR            0.053512
3             CAMPECHE            0.032787
4     DISTRITO FEDERAL            0.186275


In [26]:
!pip install folium geopandas

In [27]:
import branca.colormap as cm
import folium
import geopandas

In [28]:
df_geo = geopandas.read_file('Datos/mexican_states.geojson')
df_geo.head()

,CVE_ENT,NOM_ENT,OID,geometry
0,09,Distrito Federal,1,"POLYGON ((-98.97900 19.07446, -98.98359 19.078..."
1,12,Guerrero,2,"POLYGON ((-99.49441 18.72054, -99.47758 18.673..."
2,15,México,3,"POLYGON ((-99.85911 20.26847, -99.84189 20.281..."
3,17,Morelos,4,"POLYGON ((-98.97900 19.07446, -98.95778 19.060..."
4,25,Sinaloa,5,"POLYGON ((-106.99432 25.66223, -106.97329 25.6..."


In [29]:
incumplientos = [0.186275, 0.083333, 0.035122, 0.100503, 0.134831, 0.174051, 0.086614, 0.053512, 0.044776,
                 0.182482, 0.073394, 0.092391, 0.964286, 0.358491, 0.907801, 0.023529, 0.052632, 0.480000,
                 0.174797, 0.194175, 0.033816, 0.000000, 0.042296, 0.038462, 0.067460, 0.026178, 0.161290,
                 0.041667, 0.049763, 0.085106, 0.032787, 0.165919]

df_geo['INCUMPLIMIENTOS'] = incumplientos

# df_geo.to_file('mexican_states_info.geojson', driver='GeoJSON')

Hacemos un mapa de colores:

In [30]:
import branca.colormap as cm

colormap = cm.LinearColormap(colors = ['#12C600', '#DFE600', '#F8900B', '#E90000'],
                             index = [0, 1/3, 2/3, 1],
                             vmin = 0, vmax = 1)

colormap

In [31]:
dict_edos_cuenta = dict(zip(df_geo['NOM_ENT'], df_geo['INCUMPLIMIENTOS']))

def style_function(feature):
    return {
    'fillColor': colormap(dict_edos_cuenta[feature['properties']['NOM_ENT']]),
    'color': 'black',
    'weight': 1.5,
    }

def highlight_function(feature):
    return {
    'fillColor': colormap(dict_edos_cuenta[feature['properties']['NOM_ENT']]),
    'color': 'black',
    'weight': 3.5,
    }

In [32]:
mexico_topo_info = f'mexican_states_info.geojson'

m = folium.Map([19.42847,-99.12766], tiles='OpenStreetMap', zoom_start=5)

folium.GeoJson(
    df_geo,
    style_function = style_function,
    highlight_function = highlight_function,
    tooltip = folium.GeoJsonTooltip(
              fields=['NOM_ENT', 'INCUMPLIMIENTOS'],
              aliases=['Estado:', 'Nivel de incumplimiento: '])
).add_to(m)

colormap.caption = 'Nivel de incumplimiento'
m.add_child(colormap)

m.save('Mapas/incumplimiento.html')

m

## Recomendar productos a un usuario según lo que otros usuarios han comprado. Es decir, Dado un usuario y un producto, obtener todos los productos de esa misma categoría que han comprado otros usuarios que también han comprado ese mismo producto.

In [33]:
def recomiendaProductos(usuario_id, producto):
    with driver.session() as s:
        return s.run("""MATCH (u:Usuario) -[r1:Realiza]-> (c:Compra) -[r2:Contiene]-> (p:Producto) -[r0:Categorizado_Como]-> (cat:Categoria)
                 WHERE u.id = \"{0}\" and p.nombre = \"{1}\"
                 MATCH (u2:Usuario) -[r3:Realiza]-> (c2:Compra) -[r4:Contiene]-> (p)
                 MATCH (u2) -[r5:Realiza]-> (c3:Compra) -[r6:Contiene]-> (p2:Producto) -[r7:Categorizado_Como]-> (cat)
                 RETURN DISTINCT p2.nombre
              """.format(usuario_id, producto)).values()

In [40]:
# Todos los usarios compraron ese producto
recomiendaProductos("usuario3", "BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.")

[['BOTELLA DE TEQUILA CONTENIDO NETO 950 ML CON 35% ALC.']]